In [5]:
import numpy as np
import random
import os.path
import pickle as pkl
import torch

In [6]:
n=10
m=10

x_max=np.pi
x_min=0.0
num_x=30 

y_max=np.pi*2
y_min=0.0
num_y=30

T_max=1000.0
beta=5000.0

IC=200
train_cnt=0.75

In [7]:
dx = (x_max - x_min)/num_x
dy = (y_max - y_min)/num_y

In [57]:
x = np.append([np.arange(x_min,x_max,dx)],[x_max])

x_range = x.shape[0]
x = np.expand_dims(x, axis=0)
x = np.repeat(x,IC,axis=0)
x = np.expand_dims(x,axis=2)
c = np.zeros((IC, n))

for seed in range(1,IC+1,1):
    np.random.seed(seed)
    c[seed-1] = np.random.normal(0.0,1.0,n)

In [10]:
x = np.append([np.arange(x_min,x_max,dx)],[x_max])
x_range = x.shape[0]

y = np.append([np.arange(y_min,y_max,dy)],[y_max])
y_range = y.shape[0]


data = np.stack((x,y),axis=0)

data = np.expand_dims(data, axis=0)
data = np.repeat(data,IC,axis=0)
data = np.expand_dims(data,axis=3)
        
c = np.zeros((IC, n, m))

In [56]:
for seed in range(1,IC+1,1):
    np.random.seed(seed)
    c[seed-1] = np.random.normal(0.0,1.0,(n,m))

ValueError: could not broadcast input array from shape (10,10) into shape (10)

In [58]:
#Plain Data Gen
dt=1.0

In [59]:
data.shape

(200, 31, 1001)

In [60]:
t = np.arange(0.0,T_max+dt,dt)
t_range = t.shape[0]
t = np.expand_dims(t, axis=0)
t = np.repeat(t,IC,axis=0)

# U = np.zeros((IC,x_range,y_range, t_range))
U = np.zeros((IC,x_range, t_range))

t = np.expand_dims(t,axis=1)
t = np.repeat(t,x_range,axis=1)
# t = np.expand_dims(t,axis=1)
# t = np.repeat(t,y_range,axis=1)

# data = np.repeat(data,t_range,axis=3)
data = np.repeat(x,t_range,axis=2)

In [47]:
de= np.expand_dims(c[0,0],axis=(1,2,3))

In [61]:
data.shape

(200, 31, 1001)

In [62]:
for i in range(n):
    for j in range(m):
#         temp_c = np.expand_dims(c[0,0],axis=(1,2,3))
#         temp_c = np.repeat(np.repeat(np.repeat(temp_c,x_range,axis=1),y_range,axis=2), t_range, axis=3)
        temp_c = np.expand_dims(c[:,i],axis=(1,2))
        temp_c = np.repeat(np.repeat(temp_c,x_range,axis=1), t_range, axis=2)
        print(temp_c.shape)
        print(data.shape)
        temp_exp = np.exp(-((i+1)**2)/beta*t)
        temp_term = temp_c*np.sin((i+1)*data)
        temp = temp_term*temp_exp
        U += (1.0/n)*temp

(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)
(200, 31, 1001)


KeyboardInterrupt: 